In [ ]:
import pandas as pd
import Funciones as f
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import matplotlib.patches as mpatches
from matplotlib.patches import Rectangle

# Mapa de colores para las categorías/poblaciones
Mapa_Colores_Categorias = {
    'Progressivism': '#0078bf',
    'Moderate_Right_A': '#f7d117',
    'Moderate_Right_B': '#f7d117',
    'Left_Wing': '#f65058',
    'Blank': '#FFFFFF',
    'Centre': '#009cdd',
    'Right_Wing_Libertarian': '#753bbd'
}

In [13]:
# Cargar datos desde Excel en lugar de ejecutar todos los notebooks
import os

# Rutas a los archivos Excel
Ruta_Base = os.path.join(os.getcwd(), '..', 'Data', 'Bases definitivas')
Excel_Generales = os.path.join(Ruta_Base, 'Generales.xlsx')
Excel_Ballotage = os.path.join(Ruta_Base, 'Ballotage.xlsx')

# Cargar DataFrames desde Excel
df_Generales = pd.read_excel(Excel_Generales)
df_Ballotage = pd.read_excel(Excel_Ballotage)

# Crear diccionario de DataFrames similar al que se crea en notebooks anteriores
dfs_Finales = {
    'Generales': df_Generales,
    'Ballotage': df_Ballotage
}

print(f"✓ Datos cargados desde Excel:")
print(f"  - Generales: {len(df_Generales)} registros")
print(f"  - Ballotage: {len(df_Ballotage)} registros")

# NOTA: Los diccionarios Diccionario_Resultados_CO_Individuales y Diccionario_P_Valores
# deben cargarse desde archivos guardados previamente o recalcularse aquí.
# Por ahora, asumimos que están disponibles en el entorno.

In [14]:
def Crear_Tabla_Por_Item(
    Diccionario_Resultados,
    Diccionario_P_Valores,
    Numero_Item,
    Nombre_Archivo = None,
    Carpeta_Destino = 'Tablas_Estadisticas'
):
    """
    Crea una tabla para UN ítem específico mostrando TODAS las categorías.
    Compara Generales vs Ballotage.

    Formato:
                        | Generales                    | Ballotage                    |
                        | Media±DE  | n    | Sig       | Media±DE  | n    | Sig       |
    --------------------|-----------|------|-----------|-----------|------|-----------|
    Left_Wing (Izq)     | X.XX±Y.YY | 100  | ***       | X.XX±Y.YY | 90   | **        |
    Left_Wing (Der)     | X.XX±Y.YY | 100  | ***       | X.XX±Y.YY | 90   | **        |
    ...

    Parámetros:
    - Diccionario_Resultados: Diccionario con estadísticas por categoría
    - Diccionario_P_Valores: Diccionario con p-valores
    - Numero_Item: Número del ítem a analizar (ej: 5, 6, 9, etc.)
    - Nombre_Archivo: Nombre del archivo a guardar (si None, usa formato automático)
    - Carpeta_Destino: Carpeta donde guardar la imagen
    """

    import os
    if not os.path.exists(Carpeta_Destino):
        os.makedirs(Carpeta_Destino)

    if Nombre_Archivo is None:
        Nombre_Archivo = f'Tabla_Item_{Numero_Item}_Todas_Categorias.png'

    # Categorías a incluir
    Categorias = [
        'Left_Wing',
        'Progressivism',
        'Centre',
        'Moderate_Right_A',
        'Moderate_Right_B',
        'Right_Wing_Libertarian'
    ]

    Etiquetas_Categorias = {
        'Left_Wing': 'Left Wing',
        'Progressivism': 'Progressivism',
        'Centre': 'Centre',
        'Moderate_Right_A': 'Moderate Right A',
        'Moderate_Right_B': 'Moderate Right B',
        'Right_Wing_Libertarian': 'Right Wing Libertarian'
    }

    # Variables a buscar
    Var_Izq = f'CO_Item_{Numero_Item}_Izq'
    Var_Der = f'CO_Item_{Numero_Item}_Der'

    # Preparar datos para la tabla
    Datos_Tabla = []

    # Encabezado principal
    Fila_Principal = ['', 'Generales', '', '', 'Ballotage', '', '']
    Datos_Tabla.append(Fila_Principal)

    # Sub-encabezado
    Fila_Subencabezado = ['Categoría', 'Media±DE', 'n', 'Sig', 'Media±DE', 'n', 'Sig']
    Datos_Tabla.append(Fila_Subencabezado)

    # ELIMINADA: Línea separadora de guiones

    # Para cada categoría
    for Categoria in Categorias:
        Etiqueta = Etiquetas_Categorias.get(Categoria, Categoria)

        # Procesar Izquierda
        Fila_Izq = [f'{Etiqueta} (Izq)']

        for Dataset in ['Generales', 'Ballotage']:
            Stats = None
            P_Valor = None

            # Obtener estadísticas
            if Dataset in Diccionario_Resultados:
                if Categoria in Diccionario_Resultados[Dataset]:
                    if Var_Izq in Diccionario_Resultados[Dataset][Categoria]:
                        Stats = Diccionario_Resultados[Dataset][Categoria][Var_Izq]

            # Obtener p-valor
            if Dataset in Diccionario_P_Valores:
                if Var_Izq in Diccionario_P_Valores[Dataset]:
                    P_Valor = Diccionario_P_Valores[Dataset][Var_Izq]

            # Formatear datos
            if Stats:
                Media = Stats.get('Media', np.nan)
                DE = Stats.get('Desvio_Estandar', np.nan)
                N = Stats.get('N', 0)

                if not np.isnan(Media):
                    Texto_Media = f'{Media:.2f}±{DE:.2f}'
                    Texto_N = f'{N}'

                    # Determinar significancia
                    if P_Valor is not None and not np.isnan(P_Valor):
                        if P_Valor < 0.001:
                            Texto_Sig = '***'
                        elif P_Valor < 0.01:
                            Texto_Sig = '**'
                        elif P_Valor < 0.05:
                            Texto_Sig = '*'
                        else:
                            Texto_Sig = 'ns'
                    else:
                        Texto_Sig = '—'
                else:
                    Texto_Media = '—'
                    Texto_N = '—'
                    Texto_Sig = '—'
            else:
                Texto_Media = '—'
                Texto_N = '—'
                Texto_Sig = '—'

            Fila_Izq.extend([Texto_Media, Texto_N, Texto_Sig])

        Datos_Tabla.append(Fila_Izq)

        # Procesar Derecha
        Fila_Der = [f'{Etiqueta} (Der)']

        for Dataset in ['Generales', 'Ballotage']:
            Stats = None
            P_Valor = None

            # Obtener estadísticas
            if Dataset in Diccionario_Resultados:
                if Categoria in Diccionario_Resultados[Dataset]:
                    if Var_Der in Diccionario_Resultados[Dataset][Categoria]:
                        Stats = Diccionario_Resultados[Dataset][Categoria][Var_Der]

            # Obtener p-valor
            if Dataset in Diccionario_P_Valores:
                if Var_Der in Diccionario_P_Valores[Dataset]:
                    P_Valor = Diccionario_P_Valores[Dataset][Var_Der]

            # Formatear datos
            if Stats:
                Media = Stats.get('Media', np.nan)
                DE = Stats.get('Desvio_Estandar', np.nan)
                N = Stats.get('N', 0)

                if not np.isnan(Media):
                    Texto_Media = f'{Media:.2f}±{DE:.2f}'
                    Texto_N = f'{N}'

                    # Determinar significancia
                    if P_Valor is not None and not np.isnan(P_Valor):
                        if P_Valor < 0.001:
                            Texto_Sig = '***'
                        elif P_Valor < 0.01:
                            Texto_Sig = '**'
                        elif P_Valor < 0.05:
                            Texto_Sig = '*'
                        else:
                            Texto_Sig = 'ns'
                    else:
                        Texto_Sig = '—'
                else:
                    Texto_Media = '—'
                    Texto_N = '—'
                    Texto_Sig = '—'
            else:
                Texto_Media = '—'
                Texto_N = '—'
                Texto_Sig = '—'

            Fila_Der.extend([Texto_Media, Texto_N, Texto_Sig])

        Datos_Tabla.append(Fila_Der)

        # Línea separadora entre categorías
        Datos_Tabla.append(['', '', '', '', '', '', ''])

    # Crear figura
    Num_Filas = len(Datos_Tabla)

    Fig = plt.figure(figsize=(14, Num_Filas * 0.32))
    Ax = Fig.add_subplot(111)
    Ax.axis('tight')
    Ax.axis('off')

    # Crear tabla
    Tabla = Ax.table(
        cellText = Datos_Tabla,
        cellLoc = 'center',  # Todas las celdas centradas
        loc = 'center',
        colWidths = [0.22, 0.13, 0.08, 0.07, 0.13, 0.08, 0.07]
    )

    # Estilizar
    Tabla.auto_set_font_size(False)
    Tabla.set_fontsize(9)
    Tabla.scale(1.0, 1.4)

    # Colorear filas
    for i in range(1, Num_Filas + 1):
        if i <= len(Datos_Tabla):
            Primera_Columna = str(Datos_Tabla[i-1][0])

            # Encabezado principal (Generales / Ballotage)
            if i == 1:
                # Primera celda vacía
                Tabla[(i, 0)].set_facecolor('#2C5282')
                Tabla[(i, 0)].set_text_props(weight='bold', color='white', size=10)
                
                # Generales (columnas 1-3)
                for j in range(1, 4):
                    Tabla[(i, j)].set_facecolor('#2C5282')
                    Tabla[(i, j)].set_text_props(weight='bold', color='white', size=11, ha='center')
                
                # Ballotage (columnas 4-6)
                for j in range(4, 7):
                    Tabla[(i, j)].set_facecolor('#2C5282')
                    Tabla[(i, j)].set_text_props(weight='bold', color='white', size=11, ha='center')

            # Sub-encabezado (Media±DE, n, Sig)
            elif i == 2:
                for j in range(7):
                    Tabla[(i, j)].set_facecolor('#5B9BD5')
                    Tabla[(i, j)].set_text_props(weight='bold', color='white', size=9, ha='center')

            # Filas de datos (Izq o Der)
            elif '(Izq)' in Primera_Columna or '(Der)' in Primera_Columna:
                # Extraer nombre de categoría
                Nombre_Categoria = Primera_Columna.split(' (')[0]
                
                # Encontrar clave de categoría
                Categoria_Key = None
                for key, value in Etiquetas_Categorias.items():
                    if value == Nombre_Categoria:
                        Categoria_Key = key
                        break
                
                # Color de fondo para la categoría
                Color_Categoria = Mapa_Colores_Categorias.get(Categoria_Key, '#E7E6E6')
                
                # Primera columna con color de la categoría
                Tabla[(i, 0)].set_facecolor(Color_Categoria)
                Tabla[(i, 0)].set_text_props(weight='semibold', ha='left', color='white')

                # Colorear columnas de significancia (columnas 3 y 6)
                for j_sig in [3, 6]:
                    if j_sig < len(Datos_Tabla[i-1]):
                        Texto_Sig = str(Datos_Tabla[i-1][j_sig])

                        # Colores corregidos de significancia
                        if '***' in Texto_Sig:
                            Tabla[(i, j_sig)].set_facecolor('#90EE90')
                            Tabla[(i, j_sig)].set_text_props(weight='bold', color='#006400', ha='center')
                        elif '**' in Texto_Sig:
                            Tabla[(i, j_sig)].set_facecolor('#B8F4B8')
                            Tabla[(i, j_sig)].set_text_props(weight='bold', color='#228B22', ha='center')
                        elif '*' in Texto_Sig and not '**' in Texto_Sig:
                            Tabla[(i, j_sig)].set_facecolor('#D4F4D4')
                            Tabla[(i, j_sig)].set_text_props(weight='bold', ha='center')
                        elif 'ns' in Texto_Sig:
                            Tabla[(i, j_sig)].set_facecolor('#FFE4E1')
                            Tabla[(i, j_sig)].set_text_props(color='#8B0000', ha='center')
                        else:
                            Tabla[(i, j_sig)].set_facecolor('white')
                            Tabla[(i, j_sig)].set_text_props(ha='center')
                
                # Centrar celdas de datos (Media±DE y n)
                for j_data in [1, 2, 4, 5]:
                    Tabla[(i, j_data)].set_text_props(ha='center')

    # Título
    Titulo = f'Ítem {Numero_Item}: Cambios de Opinión por Categoría\nGenerales vs Ballotage'
    Fig.suptitle(Titulo, fontsize=14, fontweight='bold', y=0.98)

    # Leyenda
    Texto_Leyenda = (
        'Significancia: *** p<0.001 | ** p<0.01 | * p<0.05 | ns: no significativo\n'
        'Formato: Media±Desvío Estándar | n: tamaño muestral'
    )
    Fig.text(0.5, 0.02, Texto_Leyenda, ha='center', fontsize=9, style='italic')

    # Guardar
    Ruta_Completa = os.path.join(Carpeta_Destino, Nombre_Archivo)
    Fig.savefig(Ruta_Completa, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close(Fig)

    print(f"✅ Tabla del Ítem {Numero_Item} guardada en: {Ruta_Completa}")

    return Ruta_Completa

In [15]:
def Crear_Tabla_Por_Dataset(
    Diccionario_Resultados,
    Diccionario_P_Valores,
    Dataset = 'Generales',
    Items_A_Incluir = None,
    Nombre_Archivo = None,
    Carpeta_Destino = 'Tablas_Estadisticas'
):
    """
    Crea una tabla para UN dataset específico (Generales o Ballotage)
    mostrando TODAS las categorías y TODOS los ítems especificados.

    Formato:
                        | Left_Wing        | Progressivism    | Centre           | ...
                        | M±DE | n  | Sig | M±DE | n  | Sig | M±DE | n  | Sig | ...
    --------------------|------|----|----|------|----|----|------|----|----|-----
    Ítem 5 (Izq)        | X.XX | 50 | ** | X.XX | 60 | *  | X.XX | 70 | ns | ...
    Ítem 5 (Der)        | X.XX | 50 | ** | X.XX | 60 | *  | X.XX | 70 | ns | ...
    ...

    Parámetros:
    - Diccionario_Resultados: Diccionario con estadísticas
    - Diccionario_P_Valores: Diccionario con p-valores
    - Dataset: 'Generales' o 'Ballotage'
    - Items_A_Incluir: Lista de números de ítems (si None, usa todos)
    - Nombre_Archivo: Nombre del archivo (si None, usa formato automático)
    - Carpeta_Destino: Carpeta donde guardar
    """

    import os
    if not os.path.exists(Carpeta_Destino):
        os.makedirs(Carpeta_Destino)

    if Nombre_Archivo is None:
        Nombre_Archivo = f'Tabla_Dataset_{Dataset}_Todas_Categorias.png'

    # Categorías
    Categorias = [
        'Left_Wing',
        'Progressivism',
        'Centre',
        'Moderate_Right_A',
        'Moderate_Right_B',
        'Right_Wing_Libertarian'
    ]

    Etiquetas_Cortas = {
        'Left_Wing': 'LW',
        'Progressivism': 'Prog',
        'Centre': 'Ctr',
        'Moderate_Right_A': 'MRA',
        'Moderate_Right_B': 'MRB',
        'Right_Wing_Libertarian': 'RWL'
    }

    # Ítems
    if Items_A_Incluir is None:
        Items_Progresistas = [5, 6, 9, 11, 16, 20, 24, 25, 27, 28]
        Items_Conservadores = [3, 4, 7, 8, 10, 19, 22, 23, 29, 30]
        Items_A_Incluir = sorted(Items_Progresistas + Items_Conservadores)

    # Preparar datos
    Datos_Tabla = []

    # Encabezado principal - nombres de categorías (centrados)
    # Cada categoría ocupa 3 columnas, ponemos el texto en la del medio
    Fila_Categorias = ['']
    for Categoria in Categorias:
        Etiqueta = Etiquetas_Cortas.get(Categoria, Categoria)
        Fila_Categorias.extend(['', Etiqueta, ''])  # Vacío, Nombre, Vacío
    Datos_Tabla.append(Fila_Categorias)

    # Sub-encabezado - Media±DE, n, Sig
    Fila_Subencabezado = ['Ítem']
    for _ in Categorias:
        Fila_Subencabezado.extend(['M±DE', 'n', 'Sig'])
    Datos_Tabla.append(Fila_Subencabezado)

    # ELIMINADA: Línea separadora de guiones

    # Para cada ítem
    for Num_Item in Items_A_Incluir:
        # Fila para Izquierda
        Var_Izq = f'CO_Item_{Num_Item}_Izq'
        Fila_Izq = [f'Ítem {Num_Item} (Izq)']

        for Categoria in Categorias:
            Stats = None
            P_Valor = None

            # Obtener estadísticas
            if Dataset in Diccionario_Resultados:
                if Categoria in Diccionario_Resultados[Dataset]:
                    if Var_Izq in Diccionario_Resultados[Dataset][Categoria]:
                        Stats = Diccionario_Resultados[Dataset][Categoria][Var_Izq]

            # Obtener p-valor
            if Dataset in Diccionario_P_Valores:
                if Var_Izq in Diccionario_P_Valores[Dataset]:
                    P_Valor = Diccionario_P_Valores[Dataset][Var_Izq]

            # Formatear
            if Stats:
                Media = Stats.get('Media', np.nan)
                DE = Stats.get('Desvio_Estandar', np.nan)
                N = Stats.get('N', 0)

                if not np.isnan(Media):
                    Texto_Media = f'{Media:.2f}±{DE:.2f}'
                    Texto_N = f'{N}'

                    if P_Valor is not None and not np.isnan(P_Valor):
                        if P_Valor < 0.001:
                            Texto_Sig = '***'
                        elif P_Valor < 0.01:
                            Texto_Sig = '**'
                        elif P_Valor < 0.05:
                            Texto_Sig = '*'
                        else:
                            Texto_Sig = 'ns'
                    else:
                        Texto_Sig = '—'
                else:
                    Texto_Media = '—'
                    Texto_N = '—'
                    Texto_Sig = '—'
            else:
                Texto_Media = '—'
                Texto_N = '—'
                Texto_Sig = '—'

            Fila_Izq.extend([Texto_Media, Texto_N, Texto_Sig])

        Datos_Tabla.append(Fila_Izq)

        # Fila para Derecha
        Var_Der = f'CO_Item_{Num_Item}_Der'
        Fila_Der = [f'Ítem {Num_Item} (Der)']

        for Categoria in Categorias:
            Stats = None
            P_Valor = None

            # Obtener estadísticas
            if Dataset in Diccionario_Resultados:
                if Categoria in Diccionario_Resultados[Dataset]:
                    if Var_Der in Diccionario_Resultados[Dataset][Categoria]:
                        Stats = Diccionario_Resultados[Dataset][Categoria][Var_Der]

            # Obtener p-valor
            if Dataset in Diccionario_P_Valores:
                if Var_Der in Diccionario_P_Valores[Dataset]:
                    P_Valor = Diccionario_P_Valores[Dataset][Var_Der]

            # Formatear
            if Stats:
                Media = Stats.get('Media', np.nan)
                DE = Stats.get('Desvio_Estandar', np.nan)
                N = Stats.get('N', 0)

                if not np.isnan(Media):
                    Texto_Media = f'{Media:.2f}±{DE:.2f}'
                    Texto_N = f'{N}'

                    if P_Valor is not None and not np.isnan(P_Valor):
                        if P_Valor < 0.001:
                            Texto_Sig = '***'
                        elif P_Valor < 0.01:
                            Texto_Sig = '**'
                        elif P_Valor < 0.05:
                            Texto_Sig = '*'
                        else:
                            Texto_Sig = 'ns'
                    else:
                        Texto_Sig = '—'
                else:
                    Texto_Media = '—'
                    Texto_N = '—'
                    Texto_Sig = '—'
            else:
                Texto_Media = '—'
                Texto_N = '—'
                Texto_Sig = '—'

            Fila_Der.extend([Texto_Media, Texto_N, Texto_Sig])

        Datos_Tabla.append(Fila_Der)

        # Espacio entre ítems
        Fila_Vacia = ['']
        for _ in Categorias:
            Fila_Vacia.extend(['', '', ''])
        Datos_Tabla.append(Fila_Vacia)

    # Crear figura
    Num_Filas = len(Datos_Tabla)
    Num_Cols = 1 + (len(Categorias) * 3)

    # Ancho proporcional a columnas, alto proporcional a filas
    Fig = plt.figure(figsize=(20, Num_Filas * 0.25))
    Ax = Fig.add_subplot(111)
    Ax.axis('tight')
    Ax.axis('off')

    # Anchos de columnas
    Col_Widths = [0.12]  # Primera columna (ítems)
    for _ in Categorias:
        Col_Widths.extend([0.10, 0.045, 0.04])

    # Crear tabla
    Tabla = Ax.table(
        cellText = Datos_Tabla,
        cellLoc = 'center',  # Todas las celdas centradas
        loc = 'center',
        colWidths = Col_Widths
    )

    # Estilizar
    Tabla.auto_set_font_size(False)
    Tabla.set_fontsize(7.5)
    Tabla.scale(1.0, 1.3)

    # Colorear
    for i in range(1, Num_Filas + 1):
        if i <= len(Datos_Tabla):
            Primera_Columna = str(Datos_Tabla[i-1][0])

            # Encabezado de categorías (fila 1)
            if i == 1:
                # Primera celda vacía
                Tabla[(i, 0)].set_facecolor('#2C5282')
                
                # Para cada categoría, colorear sus 3 columnas con el color de la categoría
                for idx_cat, Categoria in enumerate(Categorias):
                    Color_Cat = Mapa_Colores_Categorias.get(Categoria, '#2C5282')
                    
                    for offset in range(3):  # Las 3 columnas de cada categoría
                        j = 1 + (idx_cat * 3) + offset
                        Tabla[(i, j)].set_facecolor(Color_Cat)
                        Tabla[(i, j)].set_text_props(weight='bold', color='white', size=9, ha='center')

            # Sub-encabezado (fila 2)
            elif i == 2:
                for j in range(Num_Cols):
                    Tabla[(i, j)].set_facecolor('#5B9BD5')
                    Tabla[(i, j)].set_text_props(weight='bold', color='white', size=7, ha='center')

            # Filas de ítems
            elif 'Ítem' in Primera_Columna:
                Tabla[(i, 0)].set_facecolor('#E7E6E6')
                Tabla[(i, 0)].set_text_props(weight='semibold', ha='left', size=7.5)

                # Colorear significancia (cada tercera columna)
                for idx_cat in range(len(Categorias)):
                    j_sig = 1 + (idx_cat * 3) + 2  # Columna de significancia
                    if j_sig < len(Datos_Tabla[i-1]):
                        Texto_Sig = str(Datos_Tabla[i-1][j_sig])

                        if '***' in Texto_Sig:
                            Tabla[(i, j_sig)].set_facecolor('#90EE90')
                            Tabla[(i, j_sig)].set_text_props(weight='bold', color='#006400', ha='center')
                        elif '**' in Texto_Sig:
                            Tabla[(i, j_sig)].set_facecolor('#B8F4B8')
                            Tabla[(i, j_sig)].set_text_props(weight='bold', color='#228B22', ha='center')
                        elif '*' in Texto_Sig and not '**' in Texto_Sig:
                            Tabla[(i, j_sig)].set_facecolor('#D4F4D4')
                            Tabla[(i, j_sig)].set_text_props(weight='bold', ha='center')
                        elif 'ns' in Texto_Sig:
                            Tabla[(i, j_sig)].set_facecolor('#FFE4E1')
                            Tabla[(i, j_sig)].set_text_props(color='#8B0000', ha='center')
                        else:
                            Tabla[(i, j_sig)].set_text_props(ha='center')
                
                # Centrar todas las celdas de datos
                for j in range(1, Num_Cols):
                    Tabla[(i, j)].set_text_props(ha='center')

    # Título
    Titulo = f'Cambios de Opinión - {Dataset}\nTodas las Categorías por Ítem'
    Fig.suptitle(Titulo, fontsize=16, fontweight='bold', y=0.98)

    # Leyenda
    Texto_Leyenda = (
        'Significancia: *** p<0.001 | ** p<0.01 | * p<0.05 | ns: no significativo | '
        'LW: Left Wing | Prog: Progressivism | Ctr: Centre | MRA: Moderate Right A | '
        'MRB: Moderate Right B | RWL: Right Wing Libertarian'
    )
    Fig.text(0.5, 0.01, Texto_Leyenda, ha='center', fontsize=8, style='italic', wrap=True)

    # Guardar
    Ruta_Completa = os.path.join(Carpeta_Destino, Nombre_Archivo)
    Fig.savefig(Ruta_Completa, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close(Fig)

    print(f"✅ Tabla de {Dataset} guardada en: {Ruta_Completa}")

    return Ruta_Completa

In [16]:
def Crear_Tabla_Por_Categoria(
    Diccionario_Resultados,
    Diccionario_P_Valores,
    Categoria = 'Left_Wing',
    Items_A_Incluir = None,
    Nombre_Archivo = None,
    Carpeta_Destino = 'Tablas_Estadisticas'
):
    """
    Crea una tabla para UNA categoría específica mostrando TODOS los ítems.
    Compara Generales vs Ballotage.

    Formato:
                        | Generales                    | Ballotage                    |
                        | Media±DE  | n    | Sig       | Media±DE  | n    | Sig       |
    --------------------|-----------|------|-----------|-----------|------|-----------|
    Ítem 5 (Izq)        | X.XX±Y.YY | 100  | ***       | X.XX±Y.YY | 90   | **        |
    Ítem 5 (Der)        | X.XX±Y.YY | 100  | ***       | X.XX±Y.YY | 90   | **        |
    ...

    Parámetros:
    - Diccionario_Resultados: Diccionario con estadísticas
    - Diccionario_P_Valores: Diccionario con p-valores
    - Categoria: Categoría a mostrar (ej: 'Left_Wing', 'Progressivism', etc.)
    - Items_A_Incluir: Lista de números de ítems (si None, usa todos)
    - Nombre_Archivo: Nombre del archivo (si None, usa formato automático)
    - Carpeta_Destino: Carpeta donde guardar
    """

    import os
    if not os.path.exists(Carpeta_Destino):
        os.makedirs(Carpeta_Destino)

    # Etiquetas
    Etiquetas_Categorias = {
        'Left_Wing': 'Left Wing',
        'Progressivism': 'Progressivism',
        'Centre': 'Centre',
        'Moderate_Right_A': 'Moderate Right A',
        'Moderate_Right_B': 'Moderate Right B',
        'Right_Wing_Libertarian': 'Right Wing Libertarian'
    }

    Etiqueta = Etiquetas_Categorias.get(Categoria, Categoria)

    if Nombre_Archivo is None:
        Nombre_Archivo = f'Tabla_Categoria_{Categoria}_Todos_Items.png'

    # Ítems
    if Items_A_Incluir is None:
        Items_Progresistas = [5, 6, 9, 11, 16, 20, 24, 25, 27, 28]
        Items_Conservadores = [3, 4, 7, 8, 10, 19, 22, 23, 29, 30]
        Items_A_Incluir = sorted(Items_Progresistas + Items_Conservadores)

    # Preparar datos
    Datos_Tabla = []

    # Encabezado principal
    Fila_Principal = ['', 'Generales', '', '', 'Ballotage', '', '']
    Datos_Tabla.append(Fila_Principal)

    # Sub-encabezado
    Fila_Subencabezado = ['Ítem', 'Media±DE', 'n', 'Sig', 'Media±DE', 'n', 'Sig']
    Datos_Tabla.append(Fila_Subencabezado)

    # ELIMINADA: Línea separadora de guiones

    # Para cada ítem
    for Num_Item in Items_A_Incluir:
        # Fila para Izquierda
        Var_Izq = f'CO_Item_{Num_Item}_Izq'
        Fila_Izq = [f'Ítem {Num_Item} (Izq)']

        for Dataset in ['Generales', 'Ballotage']:
            Stats = None
            P_Valor = None

            # Obtener estadísticas
            if Dataset in Diccionario_Resultados:
                if Categoria in Diccionario_Resultados[Dataset]:
                    if Var_Izq in Diccionario_Resultados[Dataset][Categoria]:
                        Stats = Diccionario_Resultados[Dataset][Categoria][Var_Izq]

            # Obtener p-valor
            if Dataset in Diccionario_P_Valores:
                if Var_Izq in Diccionario_P_Valores[Dataset]:
                    P_Valor = Diccionario_P_Valores[Dataset][Var_Izq]

            # Formatear
            if Stats:
                Media = Stats.get('Media', np.nan)
                DE = Stats.get('Desvio_Estandar', np.nan)
                N = Stats.get('N', 0)

                if not np.isnan(Media):
                    Texto_Media = f'{Media:.2f}±{DE:.2f}'
                    Texto_N = f'{N}'

                    if P_Valor is not None and not np.isnan(P_Valor):
                        if P_Valor < 0.001:
                            Texto_Sig = '***'
                        elif P_Valor < 0.01:
                            Texto_Sig = '**'
                        elif P_Valor < 0.05:
                            Texto_Sig = '*'
                        else:
                            Texto_Sig = 'ns'
                    else:
                        Texto_Sig = '—'
                else:
                    Texto_Media = '—'
                    Texto_N = '—'
                    Texto_Sig = '—'
            else:
                Texto_Media = '—'
                Texto_N = '—'
                Texto_Sig = '—'

            Fila_Izq.extend([Texto_Media, Texto_N, Texto_Sig])

        Datos_Tabla.append(Fila_Izq)

        # Fila para Derecha
        Var_Der = f'CO_Item_{Num_Item}_Der'
        Fila_Der = [f'Ítem {Num_Item} (Der)']

        for Dataset in ['Generales', 'Ballotage']:
            Stats = None
            P_Valor = None

            # Obtener estadísticas
            if Dataset in Diccionario_Resultados:
                if Categoria in Diccionario_Resultados[Dataset]:
                    if Var_Der in Diccionario_Resultados[Dataset][Categoria]:
                        Stats = Diccionario_Resultados[Dataset][Categoria][Var_Der]

            # Obtener p-valor
            if Dataset in Diccionario_P_Valores:
                if Var_Der in Diccionario_P_Valores[Dataset]:
                    P_Valor = Diccionario_P_Valores[Dataset][Var_Der]

            # Formatear
            if Stats:
                Media = Stats.get('Media', np.nan)
                DE = Stats.get('Desvio_Estandar', np.nan)
                N = Stats.get('N', 0)

                if not np.isnan(Media):
                    Texto_Media = f'{Media:.2f}±{DE:.2f}'
                    Texto_N = f'{N}'

                    if P_Valor is not None and not np.isnan(P_Valor):
                        if P_Valor < 0.001:
                            Texto_Sig = '***'
                        elif P_Valor < 0.01:
                            Texto_Sig = '**'
                        elif P_Valor < 0.05:
                            Texto_Sig = '*'
                        else:
                            Texto_Sig = 'ns'
                    else:
                        Texto_Sig = '—'
                else:
                    Texto_Media = '—'
                    Texto_N = '—'
                    Texto_Sig = '—'
            else:
                Texto_Media = '—'
                Texto_N = '—'
                Texto_Sig = '—'

            Fila_Der.extend([Texto_Media, Texto_N, Texto_Sig])

        Datos_Tabla.append(Fila_Der)

        # Espacio entre ítems
        Datos_Tabla.append(['', '', '', '', '', '', ''])

    # Crear figura
    Num_Filas = len(Datos_Tabla)

    Fig = plt.figure(figsize=(14, Num_Filas * 0.28))
    Ax = Fig.add_subplot(111)
    Ax.axis('tight')
    Ax.axis('off')

    # Crear tabla
    Tabla = Ax.table(
        cellText = Datos_Tabla,
        cellLoc = 'center',  # Todas las celdas centradas
        loc = 'center',
        colWidths = [0.18, 0.14, 0.08, 0.07, 0.14, 0.08, 0.07]
    )

    # Estilizar
    Tabla.auto_set_font_size(False)
    Tabla.set_fontsize(8.5)
    Tabla.scale(1.0, 1.35)

    # Colorear
    for i in range(1, Num_Filas + 1):
        if i <= len(Datos_Tabla):
            Primera_Columna = str(Datos_Tabla[i-1][0])

            # Encabezado principal (Generales / Ballotage)
            if i == 1:
                # Primera celda vacía
                Tabla[(i, 0)].set_facecolor('#2C5282')
                Tabla[(i, 0)].set_text_props(weight='bold', color='white', size=10)
                
                # Generales (columnas 1-3)
                for j in range(1, 4):
                    Tabla[(i, j)].set_facecolor('#2C5282')
                    Tabla[(i, j)].set_text_props(weight='bold', color='white', size=11, ha='center')
                
                # Ballotage (columnas 4-6)
                for j in range(4, 7):
                    Tabla[(i, j)].set_facecolor('#2C5282')
                    Tabla[(i, j)].set_text_props(weight='bold', color='white', size=11, ha='center')

            # Sub-encabezado (Media±DE, n, Sig)
            elif i == 2:
                # Primera celda con color de la categoría
                Color_Cat = Mapa_Colores_Categorias.get(Categoria, '#5B9BD5')
                Tabla[(i, 0)].set_facecolor(Color_Cat)
                Tabla[(i, 0)].set_text_props(weight='bold', color='white', size=9, ha='center')
                
                # Resto de celdas del sub-encabezado
                for j in range(1, 7):
                    Tabla[(i, j)].set_facecolor('#5B9BD5')
                    Tabla[(i, j)].set_text_props(weight='bold', color='white', size=9, ha='center')

            # Filas de ítems
            elif 'Ítem' in Primera_Columna:
                Tabla[(i, 0)].set_facecolor('#E7E6E6')
                Tabla[(i, 0)].set_text_props(weight='semibold', ha='left')

                # Colorear significancia (columnas 3 y 6)
                for j_sig in [3, 6]:
                    if j_sig < len(Datos_Tabla[i-1]):
                        Texto_Sig = str(Datos_Tabla[i-1][j_sig])

                        if '***' in Texto_Sig:
                            Tabla[(i, j_sig)].set_facecolor('#90EE90')
                            Tabla[(i, j_sig)].set_text_props(weight='bold', color='#006400', ha='center')
                        elif '**' in Texto_Sig:
                            Tabla[(i, j_sig)].set_facecolor('#B8F4B8')
                            Tabla[(i, j_sig)].set_text_props(weight='bold', color='#228B22', ha='center')
                        elif '*' in Texto_Sig and not '**' in Texto_Sig:
                            Tabla[(i, j_sig)].set_facecolor('#D4F4D4')
                            Tabla[(i, j_sig)].set_text_props(weight='bold', ha='center')
                        elif 'ns' in Texto_Sig:
                            Tabla[(i, j_sig)].set_facecolor('#FFE4E1')
                            Tabla[(i, j_sig)].set_text_props(color='#8B0000', ha='center')
                        else:
                            Tabla[(i, j_sig)].set_text_props(ha='center')
                
                # Centrar celdas de datos (Media±DE y n)
                for j_data in [1, 2, 4, 5]:
                    Tabla[(i, j_data)].set_text_props(ha='center')

    # Título
    Titulo = f'Cambios de Opinión - {Etiqueta}\nGenerales vs Ballotage - Todos los Ítems'
    Fig.suptitle(Titulo, fontsize=14, fontweight='bold', y=0.98)

    # Leyenda
    Texto_Leyenda = (
        'Significancia: *** p<0.001 | ** p<0.01 | * p<0.05 | ns: no significativo\n'
        'Formato: Media±Desvío Estándar | n: tamaño muestral'
    )
    Fig.text(0.5, 0.02, Texto_Leyenda, ha='center', fontsize=9, style='italic')

    # Guardar
    Ruta_Completa = os.path.join(Carpeta_Destino, Nombre_Archivo)
    Fig.savefig(Ruta_Completa, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close(Fig)

    print(f"✅ Tabla de {Etiqueta} guardada en: {Ruta_Completa}")

    return Ruta_Completa

In [17]:
def Crear_Tabla_Total_Jerarquica(
    Diccionario_Resultados,
    Diccionario_P_Valores,
    Items_A_Incluir = None,
    Nombre_Archivo = 'Tabla_Total_Jerarquica.png',
    Carpeta_Destino = 'Tablas_Estadisticas'
):
    """
    Crea UNA mega-tabla con TODO: todas las categorías, ambos datasets (Generales/Ballotage),
    y todos los ítems. Las columnas están agrupadas jerárquicamente:
    
    Nivel 1: Categorías (Left_Wing, Progressivism, etc.)
    Nivel 2: Datasets (Generales, Ballotage)
    Nivel 3: Media±DE, n, Sig

    Formato:
                    | Left_Wing                                          | Progressivism                                      | ...
                    | Generales          | Ballotage          | Generales          | Ballotage          | ...
                    | M±DE | n  | Sig    | M±DE | n  | Sig    | M±DE | n  | Sig    | M±DE | n  | Sig    | ...
    ----------------|------|----|----|------|----|----|------|----|----|------|----|----|-----
    Ítem 5 (Izq)    | X.XX | 50 | ** | X.XX | 45 | *  | X.XX | 60 | ns | X.XX | 55 | ** | ...
    Ítem 5 (Der)    | X.XX | 50 | ** | X.XX | 45 | *  | X.XX | 60 | ns | X.XX | 55 | ** | ...
    ...

    Parámetros:
    - Diccionario_Resultados: Diccionario con estadísticas
    - Diccionario_P_Valores: Diccionario con p-valores
    - Items_A_Incluir: Lista de números de ítems (si None, usa todos)
    - Nombre_Archivo: Nombre del archivo
    - Carpeta_Destino: Carpeta donde guardar
    """

    import os
    if not os.path.exists(Carpeta_Destino):
        os.makedirs(Carpeta_Destino)

    # Categorías
    Categorias = [
        'Left_Wing',
        'Progressivism',
        'Centre',
        'Moderate_Right_A',
        'Moderate_Right_B',
        'Right_Wing_Libertarian'
    ]

    Etiquetas_Cortas = {
        'Left_Wing': 'LW',
        'Progressivism': 'Prog',
        'Centre': 'Ctr',
        'Moderate_Right_A': 'MRA',
        'Moderate_Right_B': 'MRB',
        'Right_Wing_Libertarian': 'RWL'
    }

    # Ítems
    if Items_A_Incluir is None:
        Items_Progresistas = [5, 6, 9, 11, 16, 20, 24, 25, 27, 28]
        Items_Conservadores = [3, 4, 7, 8, 10, 19, 22, 23, 29, 30]
        Items_A_Incluir = sorted(Items_Progresistas + Items_Conservadores)

    # Preparar datos
    Datos_Tabla = []

    # ENCABEZADO NIVEL 1: Categorías
    # Cada categoría ocupa 6 columnas, ponemos el texto en la columna del medio (columna 3 de 6)
    Fila_Categorias = ['']
    for Categoria in Categorias:
        Etiqueta = Etiquetas_Cortas.get(Categoria, Categoria)
        # Columnas: vacío, vacío, Nombre, vacío, vacío, vacío
        Fila_Categorias.extend(['', '', Etiqueta, '', '', ''])
    Datos_Tabla.append(Fila_Categorias)

    # ENCABEZADO NIVEL 2: Datasets
    # Cada dataset ocupa 3 columnas, ponemos el texto en la columna del medio (columna 2 de 3)
    Fila_Datasets = ['']
    for _ in Categorias:
        # Para cada categoría: Gen (vacío, Gen, vacío) + Ball (vacío, Ball, vacío)
        Fila_Datasets.extend(['', 'Gen', '', '', 'Ball', ''])
    Datos_Tabla.append(Fila_Datasets)

    # ENCABEZADO NIVEL 3: Métricas
    Fila_Metricas = ['Ítem']
    for _ in Categorias:
        for _ in range(2):  # 2 datasets
            Fila_Metricas.extend(['M±DE', 'n', 'Sig'])
    Datos_Tabla.append(Fila_Metricas)

    # ELIMINADA: Línea separadora de guiones

    # DATOS: Para cada ítem
    for Num_Item in Items_A_Incluir:
        # Fila para Izquierda
        Var_Izq = f'CO_Item_{Num_Item}_Izq'
        Fila_Izq = [f'Ítem {Num_Item} (Izq)']

        for Categoria in Categorias:
            for Dataset in ['Generales', 'Ballotage']:
                Stats = None
                P_Valor = None

                # Obtener estadísticas
                if Dataset in Diccionario_Resultados:
                    if Categoria in Diccionario_Resultados[Dataset]:
                        if Var_Izq in Diccionario_Resultados[Dataset][Categoria]:
                            Stats = Diccionario_Resultados[Dataset][Categoria][Var_Izq]

                # Obtener p-valor
                if Dataset in Diccionario_P_Valores:
                    if Var_Izq in Diccionario_P_Valores[Dataset]:
                        P_Valor = Diccionario_P_Valores[Dataset][Var_Izq]

                # Formatear
                if Stats:
                    Media = Stats.get('Media', np.nan)
                    DE = Stats.get('Desvio_Estandar', np.nan)
                    N = Stats.get('N', 0)

                    if not np.isnan(Media):
                        Texto_Media = f'{Media:.2f}±{DE:.2f}'
                        Texto_N = f'{N}'

                        if P_Valor is not None and not np.isnan(P_Valor):
                            if P_Valor < 0.001:
                                Texto_Sig = '***'
                            elif P_Valor < 0.01:
                                Texto_Sig = '**'
                            elif P_Valor < 0.05:
                                Texto_Sig = '*'
                            else:
                                Texto_Sig = 'ns'
                        else:
                            Texto_Sig = '—'
                    else:
                        Texto_Media = '—'
                        Texto_N = '—'
                        Texto_Sig = '—'
                else:
                    Texto_Media = '—'
                    Texto_N = '—'
                    Texto_Sig = '—'

                Fila_Izq.extend([Texto_Media, Texto_N, Texto_Sig])

        Datos_Tabla.append(Fila_Izq)

        # Fila para Derecha
        Var_Der = f'CO_Item_{Num_Item}_Der'
        Fila_Der = [f'Ítem {Num_Item} (Der)']

        for Categoria in Categorias:
            for Dataset in ['Generales', 'Ballotage']:
                Stats = None
                P_Valor = None

                # Obtener estadísticas
                if Dataset in Diccionario_Resultados:
                    if Categoria in Diccionario_Resultados[Dataset]:
                        if Var_Der in Diccionario_Resultados[Dataset][Categoria]:
                            Stats = Diccionario_Resultados[Dataset][Categoria][Var_Der]

                # Obtener p-valor
                if Dataset in Diccionario_P_Valores:
                    if Var_Der in Diccionario_P_Valores[Dataset]:
                        P_Valor = Diccionario_P_Valores[Dataset][Var_Der]

                # Formatear
                if Stats:
                    Media = Stats.get('Media', np.nan)
                    DE = Stats.get('Desvio_Estandar', np.nan)
                    N = Stats.get('N', 0)

                    if not np.isnan(Media):
                        Texto_Media = f'{Media:.2f}±{DE:.2f}'
                        Texto_N = f'{N}'

                        if P_Valor is not None and not np.isnan(P_Valor):
                            if P_Valor < 0.001:
                                Texto_Sig = '***'
                            elif P_Valor < 0.01:
                                Texto_Sig = '**'
                            elif P_Valor < 0.05:
                                Texto_Sig = '*'
                            else:
                                Texto_Sig = 'ns'
                        else:
                            Texto_Sig = '—'
                    else:
                        Texto_Media = '—'
                        Texto_N = '—'
                        Texto_Sig = '—'
                else:
                    Texto_Media = '—'
                    Texto_N = '—'
                    Texto_Sig = '—'

                Fila_Der.extend([Texto_Media, Texto_N, Texto_Sig])

        Datos_Tabla.append(Fila_Der)

        # Espacio entre ítems
        Fila_Vacia = ['']
        for _ in Categorias:
            for _ in range(2):
                Fila_Vacia.extend(['', '', ''])
        Datos_Tabla.append(Fila_Vacia)

    # Crear figura
    Num_Filas = len(Datos_Tabla)
    Num_Cols = 1 + (len(Categorias) * 6)  # 1 para ítems + (6 cats × 6 cols cada una)

    # Tabla muy ancha
    Fig = plt.figure(figsize=(28, Num_Filas * 0.22))
    Ax = Fig.add_subplot(111)
    Ax.axis('tight')
    Ax.axis('off')

    # Anchos de columnas
    Col_Widths = [0.08]  # Primera columna (ítems)
    for _ in Categorias:
        for _ in range(2):  # 2 datasets
            Col_Widths.extend([0.065, 0.028, 0.025])

    # Crear tabla
    Tabla = Ax.table(
        cellText = Datos_Tabla,
        cellLoc = 'center',  # Todas las celdas centradas
        loc = 'center',
        colWidths = Col_Widths
    )

    # Estilizar
    Tabla.auto_set_font_size(False)
    Tabla.set_fontsize(6)
    Tabla.scale(1.0, 1.25)

    # Colorear
    for i in range(1, Num_Filas + 1):
        if i <= len(Datos_Tabla):
            Primera_Columna = str(Datos_Tabla[i-1][0])

            # Encabezado Nivel 1 (Categorías)
            if i == 1:
                # Primera celda vacía
                Tabla[(i, 0)].set_facecolor('#1F4788')
                
                # Para cada categoría, colorear sus 6 columnas con el color de la categoría
                for idx_cat, Categoria in enumerate(Categorias):
                    Color_Cat = Mapa_Colores_Categorias.get(Categoria, '#1F4788')
                    
                    for offset in range(6):  # Las 6 columnas de cada categoría
                        j = 1 + (idx_cat * 6) + offset
                        Tabla[(i, j)].set_facecolor(Color_Cat)
                        Tabla[(i, j)].set_text_props(weight='bold', color='white', size=7, ha='center')

            # Encabezado Nivel 2 (Datasets)
            elif i == 2:
                # Primera celda vacía
                Tabla[(i, 0)].set_facecolor('#2C5282')
                
                # Para cada dataset dentro de cada categoría
                for idx_cat in range(len(Categorias)):
                    for idx_dataset in range(2):  # Generales y Ballotage
                        for offset in range(3):  # Las 3 columnas de cada dataset
                            j = 1 + (idx_cat * 6) + (idx_dataset * 3) + offset
                            Tabla[(i, j)].set_facecolor('#2C5282')
                            Tabla[(i, j)].set_text_props(weight='bold', color='white', size=6.5, ha='center')

            # Encabezado Nivel 3 (Métricas)
            elif i == 3:
                for j in range(Num_Cols):
                    Tabla[(i, j)].set_facecolor('#5B9BD5')
                    Tabla[(i, j)].set_text_props(weight='bold', color='white', size=6, ha='center')

            # Filas de ítems
            elif 'Ítem' in Primera_Columna:
                Tabla[(i, 0)].set_facecolor('#E7E6E6')
                Tabla[(i, 0)].set_text_props(weight='semibold', ha='left', size=6)

                # Colorear significancia (cada tercera columna)
                for idx_cat in range(len(Categorias)):
                    for idx_dataset in range(2):  # Generales y Ballotage
                        j_sig = 1 + (idx_cat * 6) + (idx_dataset * 3) + 2
                        if j_sig < len(Datos_Tabla[i-1]):
                            Texto_Sig = str(Datos_Tabla[i-1][j_sig])

                            if '***' in Texto_Sig:
                                Tabla[(i, j_sig)].set_facecolor('#90EE90')
                                Tabla[(i, j_sig)].set_text_props(weight='bold', color='#006400', size=6, ha='center')
                            elif '**' in Texto_Sig:
                                Tabla[(i, j_sig)].set_facecolor('#B8F4B8')
                                Tabla[(i, j_sig)].set_text_props(weight='bold', color='#228B22', size=6, ha='center')
                            elif '*' in Texto_Sig and not '**' in Texto_Sig:
                                Tabla[(i, j_sig)].set_facecolor('#D4F4D4')
                                Tabla[(i, j_sig)].set_text_props(weight='bold', size=6, ha='center')
                            elif 'ns' in Texto_Sig:
                                Tabla[(i, j_sig)].set_facecolor('#FFE4E1')
                                Tabla[(i, j_sig)].set_text_props(color='#8B0000', size=6, ha='center')
                            else:
                                Tabla[(i, j_sig)].set_text_props(size=6, ha='center')
                
                # Centrar todas las celdas de datos
                for j in range(1, Num_Cols):
                    Tabla[(i, j)].set_text_props(size=6, ha='center')

    # Título
    Titulo = 'Tabla Completa: Cambios de Opinión por Ítem\nTodas las Categorías - Generales vs Ballotage'
    Fig.suptitle(Titulo, fontsize=16, fontweight='bold', y=0.99)

    # Leyenda
    Texto_Leyenda = (
        'Significancia: *** p<0.001 | ** p<0.01 | * p<0.05 | ns: no significativo | '
        'LW: Left Wing | Prog: Progressivism | Ctr: Centre | MRA: Moderate Right A | MRB: Moderate Right B | RWL: Right Wing Libertarian | '
        'Gen: Generales | Ball: Ballotage | M±DE: Media±Desvío Estándar'
    )
    Fig.text(0.5, 0.005, Texto_Leyenda, ha='center', fontsize=7, style='italic', wrap=True)

    # Guardar
    Ruta_Completa = os.path.join(Carpeta_Destino, Nombre_Archivo)
    Fig.savefig(Ruta_Completa, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close(Fig)

    print(f"✅ Tabla Total Jerárquica guardada en: {Ruta_Completa}")

    return Ruta_Completa

## Ejemplos de Uso

A continuación se muestran ejemplos de cómo usar cada función para generar diferentes tipos de tablas.

### 1. Tabla por Ítem
Genera una tabla para un ítem específico mostrando todas las categorías comparando Generales vs Ballotage.

In [18]:
# Ejemplo: Crear tabla para el Ítem 5
Ruta_Tabla_Item_5 = Crear_Tabla_Por_Item(
    Diccionario_Resultados = Diccionario_Resultados_CO_Individuales,
    Diccionario_P_Valores = Diccionario_P_Valores,
    Numero_Item = 5,
    Nombre_Archivo = 'Ejemplo_Tabla_Item_5.png'
)

print(f"\n📊 Tabla del Ítem 5 creada")
print(f"   Muestra todas las categorías comparando Generales vs Ballotage")

✅ Tabla del Ítem 5 guardada en: Tablas_Estadisticas\Ejemplo_Tabla_Item_5.png

📊 Tabla del Ítem 5 creada
   Muestra todas las categorías comparando Generales vs Ballotage


### 2. Tabla por Dataset
Genera una tabla para un dataset específico (Generales o Ballotage) mostrando todas las categorías y todos los ítems.

In [19]:
# Ejemplo: Crear tabla para Generales con todos los ítems
Ruta_Tabla_Generales = Crear_Tabla_Por_Dataset(
    Diccionario_Resultados = Diccionario_Resultados_CO_Individuales,
    Diccionario_P_Valores = Diccionario_P_Valores,
    Dataset = 'Generales',
    Items_A_Incluir = [5, 6, 9, 11, 16, 20],  # Subset de ítems progresistas
    Nombre_Archivo = 'Ejemplo_Tabla_Generales.png'
)

print(f"\n📊 Tabla de Generales creada")
print(f"   Muestra todas las categorías para 6 ítems progresistas")

✅ Tabla de Generales guardada en: Tablas_Estadisticas\Ejemplo_Tabla_Generales.png

📊 Tabla de Generales creada
   Muestra todas las categorías para 6 ítems progresistas


### 3. Tabla por Categoría
Genera una tabla para una categoría específica mostrando todos los ítems comparando Generales vs Ballotage.

In [20]:
# Ejemplo: Crear tabla para Left_Wing con todos los ítems
Ruta_Tabla_Left_Wing = Crear_Tabla_Por_Categoria(
    Diccionario_Resultados = Diccionario_Resultados_CO_Individuales,
    Diccionario_P_Valores = Diccionario_P_Valores,
    Categoria = 'Left_Wing',
    Items_A_Incluir = [5, 6, 9, 11, 16, 20, 24, 25, 27, 28],  # Ítems progresistas
    Nombre_Archivo = 'Ejemplo_Tabla_Left_Wing.png'
)

print(f"\n📊 Tabla de Left Wing creada")
print(f"   Muestra todos los ítems progresistas comparando Generales vs Ballotage")

✅ Tabla de Left Wing guardada en: Tablas_Estadisticas\Ejemplo_Tabla_Left_Wing.png

📊 Tabla de Left Wing creada
   Muestra todos los ítems progresistas comparando Generales vs Ballotage


### 4. Tabla Total Jerárquica
Genera UNA mega-tabla con TODO: todas las categorías, ambos datasets (Generales/Ballotage), y todos los ítems. Las columnas están agrupadas jerárquicamente.

In [21]:
# Ejemplo: Crear tabla total jerárquica con subset de ítems
Ruta_Tabla_Total = Crear_Tabla_Total_Jerarquica(
    Diccionario_Resultados = Diccionario_Resultados_CO_Individuales,
    Diccionario_P_Valores = Diccionario_P_Valores,
    Items_A_Incluir = [5, 6, 9, 11],  # Subset de 4 ítems para ejemplo
    Nombre_Archivo = 'Ejemplo_Tabla_Total_Jerarquica.png'
)

print(f"\n📊 Tabla Total Jerárquica creada")
print(f"   Muestra TODAS las categorías y ambos datasets para 4 ítems")
print(f"   Formato jerárquico: Categorías > Datasets > Métricas")

✅ Tabla Total Jerárquica guardada en: Tablas_Estadisticas\Ejemplo_Tabla_Total_Jerarquica.png

📊 Tabla Total Jerárquica creada
   Muestra TODAS las categorías y ambos datasets para 4 ítems
   Formato jerárquico: Categorías > Datasets > Métricas


---

## Generar Tablas Completas

Para generar todas las tablas con todos los ítems, puedes ejecutar las siguientes celdas:

In [22]:
# Generar tablas para TODOS los ítems progresistas y conservadores
Items_Progresistas = [5, 6, 9, 11, 16, 20, 24, 25, 27, 28]
Items_Conservadores = [3, 4, 7, 8, 10, 19, 22, 23, 29, 30]
Todos_Items = sorted(Items_Progresistas + Items_Conservadores)

# 1. Tablas por Dataset (una para Generales, otra para Ballotage)
print("="*60)
print("Generando Tablas por Dataset")
print("="*60)

Crear_Tabla_Por_Dataset(
    Diccionario_Resultados = Diccionario_Resultados_CO_Individuales,
    Diccionario_P_Valores = Diccionario_P_Valores,
    Dataset = 'Generales',
    Items_A_Incluir = Todos_Items,
    Nombre_Archivo = 'Tabla_Generales_Completa.png'
)

Crear_Tabla_Por_Dataset(
    Diccionario_Resultados = Diccionario_Resultados_CO_Individuales,
    Diccionario_P_Valores = Diccionario_P_Valores,
    Dataset = 'Ballotage',
    Items_A_Incluir = Todos_Items,
    Nombre_Archivo = 'Tabla_Ballotage_Completa.png'
)

# 2. Tablas por Categoría (una por cada categoría)
print("\n" + "="*60)
print("Generando Tablas por Categoría")
print("="*60)

Categorias = ['Left_Wing', 'Progressivism', 'Centre', 
              'Moderate_Right_A', 'Moderate_Right_B', 'Right_Wing_Libertarian']

for Categoria in Categorias:
    Crear_Tabla_Por_Categoria(
        Diccionario_Resultados = Diccionario_Resultados_CO_Individuales,
        Diccionario_P_Valores = Diccionario_P_Valores,
        Categoria = Categoria,
        Items_A_Incluir = Todos_Items
    )

# 3. Tabla Total Jerárquica con TODO
print("\n" + "="*60)
print("Generando Tabla Total Jerárquica")
print("="*60)

Crear_Tabla_Total_Jerarquica(
    Diccionario_Resultados = Diccionario_Resultados_CO_Individuales,
    Diccionario_P_Valores = Diccionario_P_Valores,
    Items_A_Incluir = Todos_Items,
    Nombre_Archivo = 'Tabla_Total_Jerarquica_Completa.png'
)

print("\n" + "="*60)
print("✅ TODAS LAS TABLAS GENERADAS EXITOSAMENTE")
print("="*60)

Generando Tablas por Dataset
✅ Tabla de Generales guardada en: Tablas_Estadisticas\Tabla_Generales_Completa.png
✅ Tabla de Ballotage guardada en: Tablas_Estadisticas\Tabla_Ballotage_Completa.png

Generando Tablas por Categoría
✅ Tabla de Left Wing guardada en: Tablas_Estadisticas\Tabla_Categoria_Left_Wing_Todos_Items.png
✅ Tabla de Progressivism guardada en: Tablas_Estadisticas\Tabla_Categoria_Progressivism_Todos_Items.png
✅ Tabla de Centre guardada en: Tablas_Estadisticas\Tabla_Categoria_Centre_Todos_Items.png
✅ Tabla de Moderate Right A guardada en: Tablas_Estadisticas\Tabla_Categoria_Moderate_Right_A_Todos_Items.png
✅ Tabla de Moderate Right B guardada en: Tablas_Estadisticas\Tabla_Categoria_Moderate_Right_B_Todos_Items.png
✅ Tabla de Right Wing Libertarian guardada en: Tablas_Estadisticas\Tabla_Categoria_Right_Wing_Libertarian_Todos_Items.png

Generando Tabla Total Jerárquica
✅ Tabla Total Jerárquica guardada en: Tablas_Estadisticas\Tabla_Total_Jerarquica_Completa.png

✅ TODAS LAS 